# Aging

O *aging* é uma estratificação que se realiza em um conjunto de dados com base em um escalonamento do número de dias decorridos entre a data de ocorrência de determinado evento constante dos dados em exame e uma data de interesse, geralmente a data de encerramento do exercício (31/12/YYYY). O *aging* permite obter um perfil dos dados em análise.

Para ilustrar o procedimento, vamos usar o conjunto de dados `Arfile.txt` que contém dados relativos ao contas a receber. Para termos uma idéia da estrutura do arquivo a ser importado, apresentamos a seguir os primeiros registros do mesmo.

```
S000030907702460020       13192.4220010101 
S000004194300870003         260.9720010103 
S000014319100870020        9541.2820010106 
S000045970990450020        2254.1920010110 
S000003018701390004        2286.8420010110 
S000000262400280009        3993.9020010111 
```

Este conjunto de dados contém os seguntes campos: Número da Conta, Código da Divisão, Código da Loja, Saldo ao término do exercício e Data de Vencimento.


Nosso objetivo será obter o perfil considerando o número de dias decorridos entre a Data de Vencimento e o encerramento do exercício, no caso 31/12/2001.  Primeiro apresentaremos o procedimento passo-a-passo e posteriormente com o uso da função `aging()` elaborada com o intuito de facilitar este procedimento.


Supondo que se queira estratificar os dados valores a receber até 30 dias do encerramento do exercício, 31 a 60 dias, 61 a 90 dias, 91 a 120 dias, 121 a 150 dias, 151 a 180 e faturas emitidas a mais de 180 dias, os passos a serem seguidos são os seguintes:

**Passo 1** - Importar a base de dados.  Os dados estão contidos em um arquivo de texto de formato fixo como pode ser visto no 
extrato acima. Para importar arquivos neste formato, o R dispõe da função `read.fwf()`.


In [3]:
# Define o diretório onde está o arquivo de dados a ser importado
setwd("C:\\Users\\Marcos\\Documents\\GitHub\\Usando-R-em-Auditoria\\dados")

In [4]:
# Importação do arquivo
contas_receber <- read.fwf("Arfile.txt", widths=c(11, 4, 4, 15, 8))

In [5]:
# Inspeciona a base de dados 
head(contas_receber)

           V1   V2 V3       V4       V5
1 S0000309077  246 20 13192.42 20010101
2 S0000041943   87  3   260.97 20010103
3 S0000143191   87 20  9541.28 20010106
4 S0000459709 9045 20  2254.19 20010110
5 S0000030187  139  4  2286.84 20010110
6 S0000002624   28  9  3993.90 20010111

In [6]:
# Verifica o tipo das variáveis
sapply(contas_receber, class)

       V1        V2        V3        V4        V5 
 "factor" "integer" "integer" "numeric" "integer" 

**Passo 2** - Preparar a base de dados para análise.

Neste exemplo, a prepração consiste em colocar nomes nas colunas, converter os campos relativos ao código da Divisão (`V2`) e código da Loja (`V3`) para fatores e converter o campo contendo data (`V5`) para o formato de data.

In [7]:
# Nomear as colunas
names(contas_receber) <- c("Conta", "Divisao", "Loja", "Saldo", "DataVenc")

# Converter o campo Divisão para fator
contas_receber$Divisao <- as.factor(contas_receber$Divisao)

# Converter o campo Loja para fator
contas_receber$Loja <- as.factor(contas_receber$Loja)

# Converter o campo DataVenc para um formato de data
contas_receber$DataVenc <- as.Date(as.character(contas_receber$DataVenc), "%Y%m%d")  

head(contas_receber)
sapply(contas_receber, class)

        Conta Divisao Loja    Saldo   DataVenc
1 S0000309077     246   20 13192.42 2001-01-01
2 S0000041943      87    3   260.97 2001-01-03
3 S0000143191      87   20  9541.28 2001-01-06
4 S0000459709    9045   20  2254.19 2001-01-10
5 S0000030187     139    4  2286.84 2001-01-10
6 S0000002624      28    9  3993.90 2001-01-11

    Conta   Divisao      Loja     Saldo  DataVenc 
 "factor"  "factor"  "factor" "numeric"    "Date" 

**Passo 3** - Calcular o número de dias decorridos da data de vencimento da fatura até a data de encerramento do exercício. No caso em exame 31/12/2001.  Os comandos seguintes ilustram como isso pode ser feito.

In [8]:
dias <- as.Date("2001-12-31") - contas_receber$DataVenc
dias <- as.numeric(dias)

**Passo 4** - Com base no número de dias calculados no passo anterior, classificar os registros da base de dados conforme o número dias calculado dentro do escalonamento definido pelo auditor.  No nosso exemplo, estaremos interessados em classificar os registros nos seguintes intervalos: `0-30 dias`, `31-60 dias`, `61-90 dias`, `91-120 dias`, `121-150 dias`, `151-180 dias`, `mais de 180 dias`. Isto pode ser feito com a função `cut()`.

In [9]:
aging <- cut(dias, breaks=c(0, 30, 60, 90, 120, 150, 180, Inf),
                   labels=c("0-30", "31-60", "61-90", "91-120", "121-150", "151-180", "> 180"))

**Passo 5** - Calcular o número de registros existentes em cada estrato, o valor total do faturamento em cada estrato e o faturamento médio em cada estrato.  Com os comandos a seguir realizamos esta etapa.

In [11]:
perfil <- data.frame(QTDREG = tapply(contas_receber$Saldo, aging, length),
                      TOTAL = tapply(contas_receber$Saldo, aging, sum),
                      MEDIA = tapply(contas_receber$Saldo, aging, mean))

Após esses passos, o *aging* desejado consta do objeto `perfil`, cujo conteúdo é o seguinte:

In [12]:
perfil

        QTDREG      TOTAL     MEDIA
0-30       491 4619781.17  9408.923
31-60      232 2546459.20 10976.117
61-90       17  136679.97  8039.998
91-120      15  108878.64  7258.576
121-150     11  145862.53 13260.230
151-180      7   31611.22  4515.889
> 180      216 2376875.23 11004.052

Para melhorar um pouco mais esse *aging* serão criadas duas novas colunas para indicar os percentuais que cada classe representa do total.

In [13]:
perfil <- transform(perfil, QTDREG.PERCT = round(QTDREG / sum(QTDREG) * 100, digits=4),
                            TOTAL.PERCT  = round(TOTAL / sum(TOTAL) * 100, digits=4))

perfil

        QTDREG      TOTAL     MEDIA QTDREG.PERCT TOTAL.PERCT
0-30       491 4619781.17  9408.923      49.6461     46.3547
31-60      232 2546459.20 10976.117      23.4580     25.5511
61-90       17  136679.97  8039.998       1.7189      1.3714
91-120      15  108878.64  7258.576       1.5167      1.0925
121-150     11  145862.53 13260.230       1.1122      1.4636
151-180      7   31611.22  4515.889       0.7078      0.3172
> 180      216 2376875.23 11004.052      21.8402     23.8495